In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
def R_ion_bubble(t, f_esc, N_dot_ion, n_H):
    return (3 * f_esc * N_dot_ion * t / (4 * np.pi * n_H)) ** (1 / 3)

In [ ]:
# # xi_ion for COLA1
f_esc = 0.4
# fHb = 4.06e-18
LHb = 4.06e-18 * cosmo.luminosity_distance(6.59165).to(u.cm).value**2 * np.pi * 4
L_UV = 1.3865847552558819e+29

# # Schaerer+18
N_LyC = 2.1e12 * (1 - f_esc)**-1 * LHb
print(f'{N_LyC=}')
xi_ion = N_LyC / L_UV
print(f'{np.log10(xi_ion)=}')
xi_ion = 10**25.45

# Mason&Gronke20, Wistok+23: R_ion(t) = (A * t)**(1/3)
redshift = 6.6
n_H = 1.88e-7 * (1 + redshift)**3
A = 3 * f_esc * N_LyC / (4 * np.pi * n_H) * u.s**-1 * u.cm**3

# For a bubble of 0.7 Mpc:
R_ion = 0.7 * u.Mpc
AGE = R_ion**3 / A
AGE.to(u.Myr)

In [ ]:
# For the other galaxies in z~6.6
M_UV_list = [-17.42, -19.06, -17.56, -19.3, -19.1]

fnu_other = 10**(-0.4 * (np.asarray(M_UV_list) + 48.6))  # erg/s/Hz/cm2 at 10 pc
LUV_other = np.sum(fnu_other) * (4. * np.pi * (10. * 3.08568 * 10**18)**2)  # erg/s/Hz
print(LUV_other)

# xi_ion median from Saxena+23
N_LyC_other = 10**25.59 * LUV_other

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

t_Arr = np.linspace(0, 200, 1000) * u.Myr

fesc_list = [0.05, 0.10, 0.2, 0.3, 0.4, 0.6]

for iii, f_esc in enumerate(fesc_list):
    N_LyC = 2.1e12 * (1 - f_esc)**-1 * LHb
    R_ion = R_ion_bubble(t_Arr, f_esc, N_LyC * u.s**-1, n_H * u.cm**-3)

    ax.plot(t_Arr, R_ion.to(u.Mpc).value,
            label=f'{f_esc* 100:0.0f}' + r'\%',
            lw=2.2, c=f'C{iii}')

for iii, f_esc in enumerate(fesc_list):
    N_LyC = 2.1e12 * (1 - f_esc)**-1 * LHb
    R_ion = R_ion_bubble(t_Arr, f_esc, (N_LyC + N_LyC_other / f_esc * 0.1) * u.s**-1, n_H * u.cm**-3)

    print(f'{f_esc=}, {N_LyC * f_esc / (0.1 * N_LyC_other + f_esc * N_LyC)}')

    ax.plot(t_Arr, R_ion.to(u.Mpc).value, ls=':', lw=2.2, c=f'C{iii}')

# ax.axhline(0.7, ls='--', c='dimgray', zorder=-99)
# ax.axhline(0.3, ls='-.', c='dimgray', zorder=-99)
xx = np.linspace(0, 200, 100)
ax.fill_between(xx, xx * 0, np.ones_like(xx) * 0.7, alpha=0.15,
                color='k', lw=0, zorder=-99)
ax.fill_between(xx, xx * 0, np.ones_like(xx) * 0.3, alpha=0.15,
                color='k', lw=0, zorder=-99)

ax.set_xlim(0, 100)
ax.set_ylim(0, 1.5)

# ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_ylabel('Bubble radius [pMpc]')
ax.set_xlabel(r'$t_*$ [Myr]')

# ax.plot([], [], ls='-', c='none', label=r'$f_{\rm esc}$(LyC)')

handles, labels = ax.get_legend_handles_labels()
order = np.arange(len(labels))[::-1]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
          fontsize=11, frameon=False, markerfirst=True,
          title=r'$f_{\rm esc}$(LyC)', title_fontsize=11.5)

ax.tick_params(direction='in', which='both')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/R_ion_bubble.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()